In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import zarr
import dask.array as da
import dask
from dask.diagnostics import ProgressBar
from dask.array import stats

import torch
from torch.utils.data import Dataset
import lighting.pytorch as pl
import json
from tqdm import tqdm

In [3]:
from sdofm.datasets.SDOML import ZarrIrradianceDataModuleHMI

In [5]:
hmi_path = "/mnt/sdoml/HMI.zarr"
aia_path = "/mnt/sdoml/AIA.zarr"
eve_path = "/mnt/sdoml/EVE_legacy.zarr"
components = ["Bx", "By", "Bz"]
wavelengths = ["131A", "1600A", "1700A", "171A", "193A", "211A", "304A", "335A", "94A"]
ions = [
    "C III",
    "Fe IX",
    "Fe VIII",
    "Fe X",
    "Fe XI",
    "Fe XII",
    "Fe XIII",
    "Fe XIV",
    "Fe XIX",
    "Fe XV",
    "Fe XVI",
    "Fe XVIII",
    "Fe XVI_2",
    "Fe XX",
    "Fe XX_2",
    "Fe XX_3",
    "H I",
    "H I_2",
    "H I_3",
    "He I",
    "He II",
    "He II_2",
    "He I_2",
    "Mg IX",
    "Mg X",
    "Mg X_2",
    "Ne VII",
    "Ne VIII",
    "O II",
    "O III",
    "O III_2",
    "O II_2",
    "O IV",
    "O IV_2",
    "O V",
    "O VI",
    "S XIV",
    "Si XII",
    "Si XII_2",
]
frequency = "12min"
batch_size = 32

test = ZarrIrradianceDataModuleHMI(
    hmi_path,
    aia_path,
    eve_path,
    components,
    wavelengths,
    ions,
    frequency,
    batch_size,
    num_workers=16,
    val_months=[10, 1],
    test_months=[11, 12],
    holdout_months=[],
    cache_dir="/mnt/sdoml/metadata",
)

[* CACHE SYSTEM *] Found cached index data in /mnt/sdoml/metadata/aligndata_HMI_FULL_AIA_FULL_EVE_FULL_12min.csv.
[* CACHE SYSTEM *] Found cached normalization data in /mnt/sdoml/metadata/normalizations_HMI_FULL_AIA_FULL_EVE_FULL_12min.json.


In [53]:
"_".join(["1", "3", ""])

'1_3_'

In [48]:
aia_data = zarr.group(zarr.DirectoryStore("/mnt/sdoml/AIA.zarr/"))

In [50]:
aia_data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='2010', nodes=(Node(disabled=True, i…

In [60]:
aia_data[2023]["131A"].attrs["T_OBS"][-1]

'2023-05-26T06:36:08.072'

In [45]:
data = aia_data[2010]["131A"]
exptime = np.array(data.attrs["EXPTIME"])

In [63]:
np.concatenate(None, np.zeros((0, 2)))

TypeError: dispatcher for __array_function__ did not return an iterable

In [11]:
a = [
    "C III",
    "Fe IX",
    "Fe VIII",
    "Fe X",
    "Fe XI",
    "Fe XII",
    "Fe XIII",
    "Fe XIV",
    "Fe XIX",
    "Fe XV",
    "Fe XVI",
    "Fe XVIII",
    "Fe XVI_2",
    "Fe XX",
    "Fe XX_2",
    "Fe XX_3",
    "H I",
    "H I_2",
    "H I_3",
    "He I",
    "He II",
    "He II_2",
    "He I_2",
    "Mg IX",
    "Mg X",
    "Mg X_2",
    "Ne VII",
    "Ne VIII",
    "O II",
    "O III",
    "O III_2",
    "O II_2",
    "O IV",
    "O IV_2",
    "O V",
    "O VI",
    "S XIV",
    "Si XII",
    "Si XII_2",
]
len(a)

39

In [28]:
a = np.arange(4096 * 4096).reshape(4096, 4096)
a

array([[       0,        1,        2, ...,     4093,     4094,     4095],
       [    4096,     4097,     4098, ...,     8189,     8190,     8191],
       [    8192,     8193,     8194, ...,    12285,    12286,    12287],
       ...,
       [16764928, 16764929, 16764930, ..., 16769021, 16769022, 16769023],
       [16769024, 16769025, 16769026, ..., 16773117, 16773118, 16773119],
       [16773120, 16773121, 16773122, ..., 16777213, 16777214, 16777215]])

In [31]:
from skimage.transform import downscale_local_mean

downscale_local_mean(a, (8, 8)).shape

(512, 512)

In [18]:
10 / 4

2.5

In [19]:
np.sqrt(4096)

64.0

In [27]:
32 * 32

1024